# UMAP clustering for the methylation signature Position Weight Matrices

## Load libraries and data

In [ ]:
import pandas as pd
import numpy as np
import os 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import umap
from sklearn.metrics import silhouette_score
import seaborn as sns
import os
from PIL import Image, ImageFont
from sklearn.cluster import KMeans

seed = 98
import matplotlib.pyplot as plt
import seaborn as sns

# Set font
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'STIXGeneral'


In [ ]:
path_to_images = '/scratch/project_2006608/Methylation/notebooks/UMAP_HAMBI/'

In [ ]:
file_path = '/scratch/project_2006608/Methylation/HAMBI_data/merged_data.tsv'
df = pd.read_csv(file_path, sep='\t', index_col=0, low_memory=False)

In [ ]:
print(df.shape)

## View data
### Removing the contigs that have no methyaltions (or under 50)

In [ ]:
# Remove rows where df.iloc[:, :-17] meaning no methylation detected or less than 50 detected since those are converted to 0 filled matrices
df = df.loc[(df.iloc[:, :-17] != 0).any(axis=1)]

In [ ]:
print(df.shape[0])

In [ ]:
df['Domain to species'].value_counts()

In [ ]:
df['Domain to species'].nunique(dropna = False)

In [ ]:
df['Domain to genus'].nunique(dropna = False)

In [ ]:
df['Domain to family'].nunique(dropna = False)

In [ ]:
df['Domain to order'].nunique(dropna = False)

In [ ]:
df['Domain to class'].nunique(dropna = False)

In [ ]:
df['Domain to phylum'].nunique(dropna = False)

### Number of contigs where missing the species information

In [ ]:
len(df[df['Domain to species'].isna()])

## Visualize contig lengths and mapped modifications data amount

In [ ]:
# Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/HAMBI_data/metagenomic_assembly/contigs_mod_counts.tsv'

df_mod_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(df_mod_counts.shape[0])
df_mod_counts.head()

# Append to merged_data.tsv
df_ext = df.copy()

# Reorder to match
df_ordered = df_mod_counts.loc[df_ext.index]

df_mod_counts = pd.concat([df_ext, df_ordered], axis=1)
df_mod_counts.tail()

df_mod_counts = df_mod_counts.loc[df_mod_counts['mod_count'] >= 50]
#df_mod_counts = df_mod_counts.loc[df_mod_counts['mod_count'] >= 100]
#df_mod_counts = df_mod_counts.loc[df_mod_counts['mod_count'] >= 200]
#df_mod_counts = df_mod_counts.loc[df_mod_counts['mod_count'] >= 500]

print(df_mod_counts.shape[0])

In [ ]:
# Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/HAMBI_data/metagenomic_assembly/contigs_lengths.tsv'

df_lengths = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(df_lengths.shape[0])
df_lengths.head()

In [ ]:
# Append to merged_data.tsv
df_ext = df_mod_counts.copy()

# Reorder to match
df_ordered = df_lengths.loc[df_ext.index]

df_mod_counts = pd.concat([df_ext, df_ordered], axis=1)
# transform
df_mod_counts['contig_length_sqrt'] = np.sqrt(df_mod_counts['contig_length'])
df_mod_counts.tail()
df_mod_counts['mod_count_log'] = np.log(df_mod_counts['mod_count'])
df_mod_counts.tail()

### Plot 💠

In [ ]:
n_neighbors = [20]
min_dist = [0.5]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(df_mod_counts.iloc[:, :-21])
        mod_counts_UMAP_df  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'Domain to species': df_mod_counts['Domain to species'],
            'Domain to genus': df_mod_counts['Domain to genus'],
            'element': df_mod_counts['element'],
            'contig': df_mod_counts.index,
            'mod_count':df_mod_counts['mod_count'],
            'mod_count_log':df_mod_counts['mod_count_log'],
            'contig_length_sqrt':df_mod_counts['contig_length_sqrt']
        })
        
        column = 'Domain to species'
        mod_counts_UMAP_df['Domain to species'] = mod_counts_UMAP_df['Domain to species'].fillna('NA')
        value_counts = mod_counts_UMAP_df[column].value_counts().to_dict()
        mod_counts_UMAP_df[f'{column}_with_counts'] = mod_counts_UMAP_df[column].apply(lambda x: f'{x} ({value_counts[x]})')
        mod_counts_UMAP_df['count'] = mod_counts_UMAP_df[column].map(value_counts)
        mod_counts_UMAP_df = mod_counts_UMAP_df.sort_values(by=['count'], ascending=False)
        mod_counts_UMAP_df[f'{column}_with_counts'] = pd.Categorical(
            mod_counts_UMAP_df[f'{column}_with_counts'],
            categories=mod_counts_UMAP_df[f'{column}_with_counts'].unique(),
            ordered=True
        )
        
        fig = px.scatter(mod_counts_UMAP_df, 
                            x='UMAP1', 
                            y='UMAP2', 
                            labels={f'{column}_with_counts': column},  
                            color='mod_count_log',
                            title=f' Synthetic community – UMAP with n_neighbors={n}, min_dist={m}', 
                            color_continuous_scale=px.colors.sequential.Rainbow,
                            hover_data={'Domain to species': True, 'Domain to genus': True,
                                        'element': True, 'contig': True},
                            size='contig_length_sqrt')
        title = f' Synthetic community – UMAP with n_neighbors={n}, min_dist={m}'

        fig.update_layout(font=dict(family="Times New Roman"),
            title=dict(text=title, font=dict(size=30), x=0.47, xanchor='center'),
            height=1200, width=1200, template='simple_white', showlegend=True,
            legend=dict(
                title=dict(text="title", font=dict(size=26)),
                font=dict(size=26), x=0.5, y=-0.05, traceorder="normal", xanchor='center', yanchor='top', orientation='h'),
            xaxis=dict(title_font=dict(size=26), tickfont=dict(size=24),
                       showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
            yaxis=dict(title_font=dict(size=26), tickfont=dict(size=24), showgrid=True, gridcolor='lightgray',
                       zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
        fig.show()
        fig.write_image(f'UMAP_HAMBI/UMAP_{n}_{m}_contig_mod_counts_log_contig_length_sqrt.png')
        fig.write_image(f'UMAP_HAMBI/UMAP_{n}_{m}_contig_mod_counts_log_contig_length_sqrt.svg')
        fig.write_html(f'UMAP_HAMBI/UMAP_{n}_{m}_contig_mod_counts_log_contig_length_sqrt.html')

## Create concatenated images of the sequencing logos for each cluster and species

In [ ]:
def search_images_in_subfolders(path_to_folders, index, image_types):
    """Search for images by index and type from subfolders, adding placeholders if necessary."""
    images = {}
    
    subfolders = [f for f in os.listdir(path_to_folders) if os.path.isdir(os.path.join(path_to_folders, f))]
    for img_type in image_types:
        image_found = False
        for subfolder in subfolders:
            image_path = os.path.join(path_to_folders, subfolder, 'logos', f"{index}_{img_type}.png")            
            if os.path.exists(image_path):
                try:
                    images[img_type] = Image.open(image_path)
                    image_found = True
                    break
                except Exception as e:
                    print(f"Failed to open image {image_path}: {e}")
                    images[img_type] = Image.new('RGB', (1200, 300), 'white')
                    image_found = True
                    break
        if not image_found:
            images[img_type] = Image.new('RGB', (1200, 300), 'white')
            print(f"No image found for {img_type}, using placeholder.")
    
    return images

def standardize_image_sizes(images, size=(1200, 300)):
    """Resize all images to a standard size."""
    resized_images = {}
    for key, img in images.items():
        resized_images[key] = img.resize(size)
    return resized_images

def combine_images_horizontally(images):
    """Combine images horizontally."""
    if not images:
        return None
    images = standardize_image_sizes(images)
    
    total_width = sum(img.width for img in images.values())
    max_height = max(img.height for img in images.values())

    new_im = Image.new('RGB', (total_width, max_height), 'white')
    x_offset = 0
    for img_type, img in images.items():
        new_im.paste(img, (x_offset, 0))
        x_offset += img.width
    return new_im

def combine_images_vertically(image_list):
    """Combine images vertically to form a single cluster image."""
    if not image_list or all(img is None for img in image_list):
        return None
    max_width = max(img.width for img in image_list)
    total_height = sum(img.height for img in image_list)
    
    new_im = Image.new('RGB', (max_width, total_height), 'white')
    y_offset = 0
    for img in image_list:
        new_im.paste(img, (0, y_offset))
        y_offset += img.height
    return new_im


def display_and_save_image(image, title, save_path=None):
    """Display an image using matplotlib and optionally save it to a file."""
    if image is None:
        print("No image to display or save.")
        return
    
    plt.figure(figsize=(10, 5))
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')

    if save_path:
        try:
            if image.width == 0 or image.height == 0:
                print(f"Image {title} has invalid dimensions and cannot be saved.")
            else:
                image.save(save_path)
                print(f"Saved image to {save_path}")
        except Exception as e:
            print(f"Failed to save the image {title} at {save_path}: {e}")
    plt.close()

def process_clusters_and_species_images(UMAP_df, path_to_folders, image_types, outputpath):
    """Process images based on K-means clusters and species in the DataFrame."""
    cluster_col = 'cluster'  # Use only the K-means clustering column

    for cluster in UMAP_df[cluster_col].unique():
        print(f"Processing K-means cluster {cluster}")
        cluster_df = UMAP_df[UMAP_df[cluster_col] == cluster]

        for species in cluster_df['Domain to species'].unique():
            print(f"Processing species {species} in K-means cluster {cluster}")
            species_images = []
            indexes = cluster_df[cluster_df['Domain to species'] == species].index

            for index in indexes:
                images = search_images_in_subfolders(path_to_folders, index, image_types)
                combined_horizontal_image = combine_images_horizontally(images)
                if combined_horizontal_image:
                    species_images.append(combined_horizontal_image)

            final_image = combine_images_vertically(species_images)

            if final_image:
                filename = f"kmeans_cluster_{cluster}_{species}.png"
                display_path = os.path.join(outputpath, filename)
                display_and_save_image(final_image, f"K-means Cluster {cluster} - {species}", display_path)
    print("Done")

In [ ]:
path_to_folders = '/scratch/project_2006608/Methylation/HAMBI_data/'
image_types = ['m4C_logo', 'm6A_logo', 'modified_base_logo']
outputpath = '/scratch/project_2006608/Methylation/notebooks/logos_HAMBI/'

process_clusters_and_species_images(UMAP_df, path_to_folders, image_types, outputpath)

In [ ]:
species_kmeans_counts = filt_UMAP_df.groupby('Domain to species')['cluster'].nunique()

species_kmeans_counts.plot(kind='bar', color='lightgreen')
plt.title('KMEANS')
plt.xlabel('Species')
plt.ylabel('Number of Unique KMeans Clusters')

plt.xticks(rotation=90)
plt.show()

## Redraw filtered data based on mod counts
### >=50, 100, 200, 500

In [ ]:
# Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/HAMBI_data/metagenomic_assembly/contigs_mod_counts.tsv'

df_mod_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(df_mod_counts.shape[0])
df_mod_counts.head()

# Append to merged_data.tsv
df_ext = df.copy()

# Reorder to match
df_ordered = df_mod_counts.loc[df_ext.index]

df_mod_counts = pd.concat([df_ext, df_ordered], axis=1)
df_mod_counts.tail()

df_mod_counts = df_mod_counts.loc[df_mod_counts['mod_count'] >= 50]
#df_mod_counts = df_mod_counts.loc[df_mod_counts['mod_count'] >= 100]
#df_mod_counts = df_mod_counts.loc[df_mod_counts['mod_count'] >= 200]
#df_mod_counts = df_mod_counts.loc[df_mod_counts['mod_count'] >= 500]

print(df_mod_counts.shape[0])

In [ ]:
# Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/HAMBI_data/metagenomic_assembly/contigs_lengths.tsv'

df_lengths = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(df_lengths.shape[0])
df_lengths.head()

In [ ]:
# Append to merged_data.tsv
df_ext = df_mod_counts.copy()

# Reorder to match
df_ordered = df_lengths.loc[df_ext.index]

df_mod_counts = pd.concat([df_ext, df_ordered], axis=1)
# transform
df_mod_counts['contig_length_sqrt'] = np.sqrt(df_mod_counts['contig_length'])
df_mod_counts.tail()

### Plot 💠

In [ ]:
#n_neighbors = [20, 30 ]
#min_dist = [0.01, 0.1, 0.2]

n_neighbors = [20]
min_dist = [0.5]

#n_neighbors = [20]
#min_dist = [0.4]

# 50
#color_dict = ["#FF0000", "#00FF00", "#00FFFF", "#FF00FF", "#0000FF", "#800000", "#FFFF00", "#000080", "#008000", "#808000", "#008080", "#800080", "#3bccf7", "#FF4500", "#4682B4", "#DAA520"]
# New colors
color_dict = ["#FF0000", '#3cb44b', '#ffe119', '#4363d8', '#000000', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3']

# 100
#color_dict = ["#00FF00", "#FF0000", "#00FFFF", "#000080", "#FF00FF", "#800000", "#0000FF", "#008000", "#FFFF00","#008080", "#800080", "#808000", "#3bccf7", "#4682B4", "#DAA520"]

    ## no color
#color_dict = ["#FF0000"]

# 200
#color_dict = ["#00FF00", "#FF0000", "#00FFFF", "#000080", "#FF00FF", "#008080", "#800000", "#800080", "#008000","#0000FF", "#3bccf7", "#FFFF00", "#4682B4", "#DAA520"]

# 500
#color_dict = ["#00FF00", "#FF0000", "#00FFFF", "#000080", "#008080", "#800080", "#800000", "#FF00FF", "#3bccf7","#4682B4", "#008000", "#DAA520"]

custom_colors = color_dict

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(df_mod_counts.iloc[:, :-20])
        mod_counts_UMAP_df  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'Domain to species': df_mod_counts['Domain to species'],
            'Domain to genus': df_mod_counts['Domain to genus'],
            'element': df_mod_counts['element'],
            'contig': df_mod_counts.index,
            'contig_length_sqrt': df_mod_counts['contig_length_sqrt']
        })
        
        column = 'Domain to species'
        mod_counts_UMAP_df['Domain to species'] = mod_counts_UMAP_df['Domain to species'].fillna('NA')

        value_counts = mod_counts_UMAP_df[column].value_counts().to_dict()

        mod_counts_UMAP_df[f'{column}_with_counts'] = mod_counts_UMAP_df[column].apply(lambda x: f'{x} ({value_counts[x]})')

        mod_counts_UMAP_df['count'] = mod_counts_UMAP_df[column].map(value_counts)
        mod_counts_UMAP_df = mod_counts_UMAP_df.sort_values(by=['count'], ascending=False)

        mod_counts_UMAP_df[f'{column}_with_counts'] = pd.Categorical(
            mod_counts_UMAP_df[f'{column}_with_counts'],
            categories=mod_counts_UMAP_df[f'{column}_with_counts'].unique(),
            ordered=True
        )
        
        fig = px.scatter(mod_counts_UMAP_df, 
                            x='UMAP1', 
                            y='UMAP2', 
                            labels={f'{column}_with_counts': column},
                            color=f'{column}_with_counts', 
                            title=f' Synthetic community – UMAP with n_neighbors={n}, min_dist={m}',
                            color_discrete_sequence=custom_colors,
                            opacity=0.95,
                            hover_data={'Domain to species': True, 'Domain to genus': True,
                                        'element': True, 'contig': True},
                        size = 'contig_length_sqrt'
        )
        title = f' Synthetic community – UMAP with n_neighbors={n}, min_dist={m}'

        fig.update_layout(font=dict(family="Times New Roman"),
            title=dict(text=title, font=dict(size=30), x=0.47, xanchor='center'),
            height=1700, width=1200, template='simple_white', showlegend=True,
            legend=dict(
                font=dict(size=16), x=0.5, y=-0.05, traceorder="normal", xanchor='center', yanchor='top', orientation='h'),
            xaxis=dict(title_font=dict(size=26), tickfont=dict(size=24),
                       showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
            yaxis=dict(title_font=dict(size=26), tickfont=dict(size=24), showgrid=True, gridcolor='lightgray',
                       zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
        
        fig.show()
        #fig.write_image(f'UMAP_HAMBI/UMAP_{n}_{m}_above50_mod_counts.png')
        #fig.write_image(f'UMAP_HAMBI/UMAP_{n}_{m}_above50_mod_counts.svg')
        #fig.write_html(f'UMAP_HAMBI/UMAP_{n}_{m}_above50_mod_counts.html')
        
        #fig.write_image(f'UMAP_HAMBI/UMAP_{n}_{m}_above100_mod_counts.png')
        #fig.write_html(f'UMAP_HAMBI/UMAP_{n}_{m}_above100_mod_counts.html')
        #fig.write_image(f'UMAP_HAMBI/UMAP_{n}_{m}_above200_mod_counts.png')
        #fig.write_html(f'UMAP_HAMBI/UMAP_{n}_{m}_above200_mod_counts.html')
        #fig.write_image(f'UMAP_HAMBI/UMAP_{n}_{m}_above500_mod_counts.png')
        #fig.write_html(f'UMAP_HAMBI/UMAP_{n}_{m}_above500_mod_counts.html')

### Zoom into

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_zoomed = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= -6) & (mod_counts_UMAP_df['UMAP1']<= 1)
    & (mod_counts_UMAP_df['UMAP2']>= -6) & (mod_counts_UMAP_df['UMAP2']<= 1)]

# Check
mod_counts_UMAP_df_zoomed.head()

# Save those contig IDs
mod_counts_UMAP_df_zoomed.index.to_list()

df_mod_counts_zoomed = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_zoomed.index)]

### Plot zoomed

In [ ]:
#n_neighbors = [20, 30 ]
#min_dist = [0.01, 0.1, 0.2]

#n_neighbors = [20]
#min_dist = [0.5]

n_neighbors = [20]
min_dist = [0.1]

color_dict = ['#FF0000', '#008080', '#911eb4', '#9a6324','#000000']


custom_colors = color_dict

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(df_mod_counts_zoomed.iloc[:, :-20])
        mod_counts_UMAP_df  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'Domain to species': df_mod_counts_zoomed['Domain to species'],
            'Domain to genus': df_mod_counts_zoomed['Domain to genus'],
            'element': df_mod_counts_zoomed['element'],
            'contig': df_mod_counts_zoomed.index,
            'contig_length_sqrt': df_mod_counts_zoomed['contig_length_sqrt']
        })
        
        column = 'Domain to species'
        mod_counts_UMAP_df['Domain to species'] = mod_counts_UMAP_df['Domain to species'].fillna('NA')

        value_counts = mod_counts_UMAP_df[column].value_counts().to_dict()

        mod_counts_UMAP_df[f'{column}_with_counts'] = mod_counts_UMAP_df[column].apply(lambda x: f'{x} ({value_counts[x]})')

        mod_counts_UMAP_df['count'] = mod_counts_UMAP_df[column].map(value_counts)
        mod_counts_UMAP_df = mod_counts_UMAP_df.sort_values(by=['count'], ascending=False)

        mod_counts_UMAP_df[f'{column}_with_counts'] = pd.Categorical(
            mod_counts_UMAP_df[f'{column}_with_counts'],
            categories=mod_counts_UMAP_df[f'{column}_with_counts'].unique(),
            ordered=True
        )
        
        fig = px.scatter(mod_counts_UMAP_df, 
                            x='UMAP1', 
                            y='UMAP2', 
                            labels={f'{column}_with_counts': column},
                            color=f'{column}_with_counts', 
                            title=f' Synthetic community – UMAP with n_neighbors={n}, min_dist={m}',
                            color_discrete_sequence=custom_colors,
                            opacity=0.95,
                            hover_data={'Domain to species': True, 'Domain to genus': True,
                                        'element': True, 'contig': True},
                        size = 'contig_length_sqrt'
        )
        title = f' Synthetic community – UMAP with n_neighbors={n}, min_dist={m}'

        fig.update_layout(font=dict(family="Times New Roman"),
            title=dict(text=title, font=dict(size=30), x=0.47, xanchor='center'),
            height=1700, width=1200, template='simple_white', showlegend=True,
            legend=dict(
                font=dict(size=16), x=0.5, y=-0.05, traceorder="normal", xanchor='center', yanchor='top', orientation='h'),
            xaxis=dict(title_font=dict(size=26), tickfont=dict(size=24),
                       showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
            yaxis=dict(title_font=dict(size=26), tickfont=dict(size=24), showgrid=True, gridcolor='lightgray',
                       zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
        
        fig.show()
        #fig.write_image(f'UMAP_HAMBI/UMAP_{n}_{m}_above50_mod_counts_zoomed.png')
        #fig.write_image(f'UMAP_HAMBI/UMAP_{n}_{m}_above50_mod_counts_zoomed.svg')
        #fig.write_html(f'UMAP_HAMBI/UMAP_{n}_{m}_above50_mod_counts_zoomed.html')

## Save data

In [ ]:
# Check
df_mod_counts.head()
mod_counts_UMAP_df.head()

In [ ]:
# Save data
df_mod_counts.to_csv('UMAP_HAMBI_above50/df_mod_counts.csv', sep='\t', index=True)
mod_counts_UMAP_df.to_csv('UMAP_HAMBI_above50/mod_counts_UMAP_df.csv', sep='\t', index=True)

## Draw mod counts
### > 50 mod counts

In [ ]:
df_mod_counts['mod_count_log'] = np.log(df_mod_counts['mod_count'])
df_mod_counts.head()

### Plot 💠

In [ ]:
n_neighbors = [20]
min_dist = [0.5]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(df_mod_counts.iloc[:, :-21])
        mod_counts_UMAP_df  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'Domain to species': df_mod_counts['Domain to species'],
            'Domain to genus': df_mod_counts['Domain to genus'],
            'element': df_mod_counts['element'],
            'contig': df_mod_counts.index,
            'mod_count':df_mod_counts['mod_count'],
            'mod_count_log':df_mod_counts['mod_count_log'],
            'contig_length_sqrt':df_mod_counts['contig_length_sqrt']
        })
        
        fig = px.scatter(mod_counts_UMAP_df, 
                            x='UMAP1', 
                            y='UMAP2', 
                            labels={f'{column}_with_counts': column},
                            color='mod_count_log',
                            title=f' HAMBI - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_continuous_scale=px.colors.sequential.Rainbow,
                            hover_data={'Domain to species': True, 'Domain to genus': True,
                                        'element': True, 'contig': True, 'mod_count': True},
                            size='contig_length_sqrt')
        title = f' HAMBI - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()
        #fig.write_image(f'UMAP_HAMBI/UMAP_{n}_{m}_filtered_by_50_mod_counts_color_mod.png')
        #fig.write_html(f'UMAP_HAMBI/UMAP_{n}_{m}_filtered_by_50_mod_counts_color_mod.html')

## Read in and plot

In [ ]:
# Read data
df_mod_counts = pd.read_csv('UMAP_HAMBI_above50/df_mod_counts.csv', sep='\t', index_col=0, low_memory=False)
mod_counts_UMAP_df = pd.read_csv('UMAP_HAMBI_above50/mod_counts_UMAP_df.csv', sep='\t', index_col=0, low_memory=False)

# Check
df_mod_counts.head()
mod_counts_UMAP_df.head()

## Extract clusters and redraw UMAP with subset of data
### > 50 mod counts
#### C1

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C1 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= 7.4) & (mod_counts_UMAP_df['UMAP1']<= 7.7)
    & (mod_counts_UMAP_df['UMAP2']>= 3.9) & (mod_counts_UMAP_df['UMAP2']<= 4.5)]

# Check
mod_counts_UMAP_df_C1.head()

# Save those contig IDs
mod_counts_UMAP_df_C1.index.to_list()

df_mod_counts_C1 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C1.index)]

In [ ]:
# Save contig IDs
mod_counts_C1_contigs = df_mod_counts_C1.index.to_list()

directory = 'UMAP_HAMBI_above50'
file_path = os.path.join(directory, 'mod_counts_C1_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C1_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 50 mod counts
#### C2

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C2 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= -6) & (mod_counts_UMAP_df['UMAP1']<= -5.6)
    & (mod_counts_UMAP_df['UMAP2']>= -0.6) & (mod_counts_UMAP_df['UMAP2']<= 0)]

# Check
mod_counts_UMAP_df_C2.head()

# Save those contig IDs
mod_counts_UMAP_df_C2.index.to_list()

df_mod_counts_C2 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C2.index)]

In [ ]:
# Save contig IDs
mod_counts_C2_contigs = df_mod_counts_C2.index.to_list()

directory = 'UMAP_HAMBI_above50'
file_path = os.path.join(directory, 'mod_counts_C2_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C2_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 50 mod counts
#### C3

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C3 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= -7) & (mod_counts_UMAP_df['UMAP1']<= -5.6)
    & (mod_counts_UMAP_df['UMAP2']>= -2) & (mod_counts_UMAP_df['UMAP2']<= -0.8)]

# Check
mod_counts_UMAP_df_C3.head()

# Save those contig IDs
mod_counts_UMAP_df_C3.index.to_list()

df_mod_counts_C3 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C3.index)]

In [ ]:
# Save contig IDs
mod_counts_C3_contigs = df_mod_counts_C3.index.to_list()

directory = 'UMAP_HAMBI_above50'
file_path = os.path.join(directory, 'mod_counts_C3_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C3_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 50 mod counts
#### C4

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C4 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= -8) & (mod_counts_UMAP_df['UMAP1']<= -7)
    & (mod_counts_UMAP_df['UMAP2']>= 0.6) & (mod_counts_UMAP_df['UMAP2']<= 1.2)]

# Check
mod_counts_UMAP_df_C4.head()

# Save those contig IDs
mod_counts_UMAP_df_C4.index.to_list()

df_mod_counts_C4 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C4.index)]

In [ ]:
# Save contig IDs
mod_counts_C4_contigs = df_mod_counts_C4.index.to_list()

directory = 'UMAP_HAMBI_above50'
file_path = os.path.join(directory, 'mod_counts_C4_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C4_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 50 mod counts
#### C11

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C11 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= -2.5) & (mod_counts_UMAP_df['UMAP1']<= -1.5)
    & (mod_counts_UMAP_df['UMAP2']>= 4.2) & (mod_counts_UMAP_df['UMAP2']<= 4.8)]

# Check
mod_counts_UMAP_df_C11.head()

# Save those contig IDs
mod_counts_UMAP_df_C11.index.to_list()

df_mod_counts_C11 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C11.index)]

In [ ]:
# Save contig IDs
mod_counts_C11_contigs = df_mod_counts_C11.index.to_list()

directory = 'UMAP_HAMBI_above50'
file_path = os.path.join(directory, 'mod_counts_C11_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C11_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 50 mod counts
#### C6

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C6 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= -2.5) & (mod_counts_UMAP_df['UMAP1']<= -1.5)
    & (mod_counts_UMAP_df['UMAP2']>= -4.1) & (mod_counts_UMAP_df['UMAP2']<= -3)]

# Check
mod_counts_UMAP_df_C6.head()

# Save those contig IDs
mod_counts_UMAP_df_C6.index.to_list()

df_mod_counts_C6 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C6.index)]

In [ ]:
# Save contig IDs
mod_counts_C6_contigs = df_mod_counts_C6.index.to_list()

directory = 'UMAP_HAMBI_above50'
file_path = os.path.join(directory, 'mod_counts_C6_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C6_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 50 mod counts
#### C5

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C5 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= -2.5) & (mod_counts_UMAP_df['UMAP1']<= -2)
    & (mod_counts_UMAP_df['UMAP2']>= -2.1) & (mod_counts_UMAP_df['UMAP2']<= -1.5)]

# Check
mod_counts_UMAP_df_C5.head()

# Save those contig IDs
mod_counts_UMAP_df_C5.index.to_list()

df_mod_counts_C5 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C5.index)]

In [ ]:
# Save contig IDs
mod_counts_C5_contigs = df_mod_counts_C5.index.to_list()

directory = 'UMAP_HAMBI_above50'
file_path = os.path.join(directory, 'mod_counts_C5_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C5_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 50 mod counts
#### C7

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C7 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= -2) & (mod_counts_UMAP_df['UMAP1']<= -1.36)
    & (mod_counts_UMAP_df['UMAP2']>= -2.271) & (mod_counts_UMAP_df['UMAP2']<= -1.79)]

# Check
mod_counts_UMAP_df_C7.head()

# Save those contig IDs
mod_counts_UMAP_df_C7.index.to_list()

df_mod_counts_C7 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C7.index)]

In [ ]:
# Save contig IDs
mod_counts_C7_contigs = df_mod_counts_C7.index.to_list()

directory = 'UMAP_HAMBI_above50'
file_path = os.path.join(directory, 'mod_counts_C7_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C7_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 50 mod counts
#### C8

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C8 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= -3.5) & (mod_counts_UMAP_df['UMAP1']<= -2.5)
    & (mod_counts_UMAP_df['UMAP2']>= -3) & (mod_counts_UMAP_df['UMAP2']<= -2)]

# Check
mod_counts_UMAP_df_C8.head()

# Save those contig IDs
mod_counts_UMAP_df_C8.index.to_list()

df_mod_counts_C8 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C8.index)]

In [ ]:
# Save contig IDs
mod_counts_C8_contigs = df_mod_counts_C8.index.to_list()

directory = 'UMAP_HAMBI_above50'
file_path = os.path.join(directory, 'mod_counts_C8_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C8_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 50 mod counts
#### C9

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C9 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= -1.5) & (mod_counts_UMAP_df['UMAP1']<= -1.2)
    & (mod_counts_UMAP_df['UMAP2']>= -2.4) & (mod_counts_UMAP_df['UMAP2']<= -2.1)]

# Check
mod_counts_UMAP_df_C9.head()

# Save those contig IDs
mod_counts_UMAP_df_C9.index.to_list()

df_mod_counts_C9 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C9.index)]

In [ ]:
# Save contig IDs
mod_counts_C9_contigs = df_mod_counts_C9.index.to_list()

directory = 'UMAP_HAMBI_above50'
file_path = os.path.join(directory, 'mod_counts_C9_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C9_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 50 mod counts
#### C10

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C10 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= 3) & (mod_counts_UMAP_df['UMAP1']<= 5)
    & (mod_counts_UMAP_df['UMAP2']>= -4) & (mod_counts_UMAP_df['UMAP2']<= -1)]

# Check
mod_counts_UMAP_df_C10.head()

# Save those contig IDs
mod_counts_UMAP_df_C10.index.to_list()

df_mod_counts_C10 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C10.index)]

In [ ]:
# Save contig IDs
mod_counts_C10_contigs = df_mod_counts_C10.index.to_list()

directory = 'UMAP_HAMBI_above50'
file_path = os.path.join(directory, 'mod_counts_C10_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C10_contigs:
        file.write(f"{item}\n")

#### Draw with > 50 again without NA

In [ ]:
df_mod_counts_no_NA = df_mod_counts[df_mod_counts['s'].notna()]
print(df_mod_counts_no_NA)

In [ ]:
n_neighbors = [20]
min_dist = [0.5]

# 50
color_dict = ["#00FF00", "#00FFFF", "#FF00FF", "#0000FF", "#800000",
    "#FFFF00", "#000080", "#008000", "#808000", "#008080", "#800080", "#3bccf7", "#FF4500", "#4682B4", "#DAA520"]


custom_colors = color_dict
for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(df_mod_counts_no_NA.iloc[:, :-20])
        UMAP_df_mod_counts_no_NA  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'Domain to species': df_mod_counts_no_NA['Domain to species'],
            'Domain to genus': df_mod_counts_no_NA['Domain to genus'],
            'element': df_mod_counts_no_NA['element'],
            'contig': df_mod_counts_no_NA.index,
            'contig_length_sqrt': df_mod_counts_no_NA['contig_length_sqrt']
        })
        
        column = 'Domain to species'
        UMAP_df_mod_counts_no_NA['Domain to species'] = UMAP_df_mod_counts_no_NA['Domain to species'].fillna('NA')

        value_counts = UMAP_df_mod_counts_no_NA[column].value_counts().to_dict()

        UMAP_df_mod_counts_no_NA[f'{column}_with_counts'] = UMAP_df_mod_counts_no_NA[column].apply(lambda x: f'{x} ({value_counts[x]})')

        UMAP_df_mod_counts_no_NA['count'] = UMAP_df_mod_counts_no_NA[column].map(value_counts)
        UMAP_df_mod_counts_no_NA = UMAP_df_mod_counts_no_NA.sort_values(by=['count'], ascending=False)

        UMAP_df_mod_counts_no_NA[f'{column}_with_counts'] = pd.Categorical(
            UMAP_df_mod_counts_no_NA[f'{column}_with_counts'],
            categories=UMAP_df_mod_counts_no_NA[f'{column}_with_counts'].unique(),
            ordered=True
        )
        
        fig = px.scatter(UMAP_df_mod_counts_no_NA, 
                            x='UMAP1', 
                            y='UMAP2', 
                            labels={f'{column}_with_counts': column},
                            color=f'{column}_with_counts', 
                            title=f' HAMBI - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_discrete_sequence=custom_colors,
                            hover_data={'Domain to species': True, 'Domain to genus': True,
                                        'element': True, 'contig': True},
                        size = 'contig_length_sqrt')
        title = f' HAMBI - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.1,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()
        fig.write_image(f'UMAP_HAMBI/UMAP_{n}_{m}_filtered_by_50_mod_counts_no_NA.png')
        fig.write_html(f'UMAP_HAMBI/UMAP_{n}_{m}_filtered_by_50_mod_counts_no_NA.html')

## Extract clusters and redraw UMAP with subset of data
### > 100 mod counts
#### C1

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C1 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= -2.2) & (mod_counts_UMAP_df['UMAP1']<= -1.6)
    & (mod_counts_UMAP_df['UMAP2']>= 6.9) & (mod_counts_UMAP_df['UMAP2']<= 7.2)]

# Check
mod_counts_UMAP_df_C1.head()

# Save those contig IDs
mod_counts_UMAP_df_C1.index.to_list()

df_mod_counts_C1 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C1.index)]

In [ ]:
# Save contig IDs
mod_counts_C1_contigs = df_mod_counts_C1.index.to_list()

directory = 'UMAP_HAMBI_above100'
file_path = os.path.join(directory, 'mod_counts_C1_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C1_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 100 mod counts
#### C2

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C2 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= 11.6) & (mod_counts_UMAP_df['UMAP1']<= 12)
    & (mod_counts_UMAP_df['UMAP2']>= -0.8) & (mod_counts_UMAP_df['UMAP2']<= -0.6)]

# Check
mod_counts_UMAP_df_C2.head()

# Save those contig IDs
mod_counts_UMAP_df_C2.index.to_list()

df_mod_counts_C2 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C2.index)]

In [ ]:
# Save contig IDs
mod_counts_C2_contigs = df_mod_counts_C2.index.to_list()

directory = 'UMAP_HAMBI_above100'
file_path = os.path.join(directory, 'mod_counts_C2_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C2_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 100 mod counts
#### C3

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C3 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= 13.05) & (mod_counts_UMAP_df['UMAP1']<= 13.8)
    & (mod_counts_UMAP_df['UMAP2']>= -2.8) & (mod_counts_UMAP_df['UMAP2']<= -1.8)]

# Check
mod_counts_UMAP_df_C3.head()

# Save those contig IDs
mod_counts_UMAP_df_C3.index.to_list()

df_mod_counts_C3 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C3.index)]

In [ ]:
# Save contig IDs
mod_counts_C3_contigs = df_mod_counts_C3.index.to_list()

directory = 'UMAP_HAMBI_above100'
file_path = os.path.join(directory, 'mod_counts_C3_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C3_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 100 mod counts
#### C4

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C4 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= 9.9) & (mod_counts_UMAP_df['UMAP1']<= 12)
    & (mod_counts_UMAP_df['UMAP2']>= -2.5) & (mod_counts_UMAP_df['UMAP2']<= -1)]

# Check
mod_counts_UMAP_df_C4.head()

# Save those contig IDs
mod_counts_UMAP_df_C4.index.to_list()

df_mod_counts_C4 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C4.index)]

In [ ]:
# Save contig IDs
mod_counts_C4_contigs = df_mod_counts_C4.index.to_list()

directory = 'UMAP_HAMBI_above100'
file_path = os.path.join(directory, 'mod_counts_C4_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C4_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 100 mod counts
#### C5

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C5 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= 1) & (mod_counts_UMAP_df['UMAP1']<= 4.1)
    & (mod_counts_UMAP_df['UMAP2']>= 2) & (mod_counts_UMAP_df['UMAP2']<= 3.5)]

# Check
mod_counts_UMAP_df_C5.head()

# Save those contig IDs
mod_counts_UMAP_df_C5.index.to_list()

df_mod_counts_C5 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C5.index)]

In [ ]:
# Save contig IDs
mod_counts_C5_contigs = df_mod_counts_C5.index.to_list()

directory = 'UMAP_HAMBI_above100'
file_path = os.path.join(directory, 'mod_counts_C5_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C5_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 100 mod counts
#### C6

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C6 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= 7.8) & (mod_counts_UMAP_df['UMAP1']<= 8.6)
    & (mod_counts_UMAP_df['UMAP2']>= -4.4) & (mod_counts_UMAP_df['UMAP2']<= -3.4)]

# Check
mod_counts_UMAP_df_C6.head()

# Save those contig IDs
mod_counts_UMAP_df_C6.index.to_list()

df_mod_counts_C6 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C6.index)]

In [ ]:
# Save contig IDs
mod_counts_C6_contigs = df_mod_counts_C6.index.to_list()

directory = 'UMAP_HAMBI_above100'
file_path = os.path.join(directory, 'mod_counts_C6_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C6_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 100 mod counts
#### C7

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_C7_1 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= 6.8) & (mod_counts_UMAP_df['UMAP1']<= 7.5)
    & (mod_counts_UMAP_df['UMAP2']>= -4) & (mod_counts_UMAP_df['UMAP2']<= -2.8)]

mod_counts_UMAP_df_C7_2 = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= 5.35) & (mod_counts_UMAP_df['UMAP1']<= 6.5)
    & (mod_counts_UMAP_df['UMAP2']>= -3) & (mod_counts_UMAP_df['UMAP2']<= -1.97)]

mod_counts_UMAP_df_C7 = pd.concat([mod_counts_UMAP_df_C7_1, mod_counts_UMAP_df_C7_2], ignore_index=False)

# Check
mod_counts_UMAP_df_C7.head()

# Save those contig IDs
mod_counts_UMAP_df_C7.index.to_list()

df_mod_counts_C7 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_C7.index)]

In [ ]:
# Save contig IDs
mod_counts_C7_contigs = df_mod_counts_C7.index.to_list()

directory = 'UMAP_HAMBI_above100'
file_path = os.path.join(directory, 'mod_counts_C7_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C7_contigs:
        file.write(f"{item}\n")

### Focus on distinct clusters

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_focused = mod_counts_UMAP_df.loc[(mod_counts_UMAP_df['UMAP1']>= 7) & (mod_counts_UMAP_df['UMAP1']<= 9.6)
    & (mod_counts_UMAP_df['UMAP2']>= -6.1) & (mod_counts_UMAP_df['UMAP2']<= -3)]

# Check
mod_counts_UMAP_df_focused.head()

mod_counts_data_focused = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_focused['contig'])]
print(mod_counts_data_focused)

In [ ]:
#n_neighbors = [20, 30 ]
#min_dist = [0.01, 0.1, 0.2]

n_neighbors = [20]
min_dist = [0.5]

# 100
color_dict = ["#FF0000", "#008080", "#800000", "#3bccf7", "#0000FF"]

custom_colors = color_dict

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(mod_counts_data_focused.iloc[:, :-20])
        mod_counts_UMAP_df_focused  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'Domain to species': mod_counts_data_focused['Domain to species'],
            'Domain to genus': mod_counts_data_focused['Domain to genus'],
            'element': mod_counts_data_focused['element'],
            'contig': mod_counts_data_focused.index,
            'contig_length_sqrt': mod_counts_data_focused['contig_length_sqrt']
        })
        # clustering k defined choosing the k by looking at the UMAP 
        kmeans = KMeans(n_clusters=11, random_state=seed)
        
        k_labels = kmeans.fit_predict(embedding)
        
        mod_counts_UMAP_df_focused['cluster'] = k_labels
        
        column = 'Domain to species'
        mod_counts_UMAP_df_focused['Domain to species'] = mod_counts_UMAP_df_focused['Domain to species'].fillna('NA')

        value_counts = mod_counts_UMAP_df_focused[column].value_counts().to_dict()

        mod_counts_UMAP_df_focused[f'{column}_with_counts'] = mod_counts_UMAP_df_focused[column].apply(lambda x: f'{x} ({value_counts[x]})')

        mod_counts_UMAP_df_focused['count'] = mod_counts_UMAP_df_focused[column].map(value_counts)
        mod_counts_UMAP_df_focused = mod_counts_UMAP_df_focused.sort_values(by=['count'], ascending=False)

        mod_counts_UMAP_df_focused[f'{column}_with_counts'] = pd.Categorical(
            mod_counts_UMAP_df_focused[f'{column}_with_counts'],
            categories=mod_counts_UMAP_df_focused[f'{column}_with_counts'].unique(),
            ordered=True
        )
        
        fig = px.scatter(mod_counts_UMAP_df_focused, 
                            x='UMAP1', 
                            y='UMAP2', 
                            labels={f'{column}_with_counts': column},
                            color=f'{column}_with_counts', 
                            title=f' HAMBI - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_discrete_sequence=custom_colors,
                            hover_data={'cluster': True, 'Domain to species': True, 'Domain to genus': True,
                                        'element': True, 'contig': True},
                        size = 'contig_length_sqrt'
                        )
        title = f' HAMBI - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.1,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()
        #fig.write_image(f'UMAP_HAMBI/UMAP_{n}_{m}_filtered_by_100_mod_counts_focused.png')
        #fig.write_html(f'UMAP_HAMBI/UMAP_{n}_{m}_filtered_by_100_mod_counts_focused.html')

## Extract clusters and redraw UMAP with subset of data
### > 100 mod counts
#### C8

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_focused_C8 = mod_counts_UMAP_df_focused.loc[(mod_counts_UMAP_df_focused['UMAP1']>= 10) & (mod_counts_UMAP_df_focused['UMAP1']<= 14)
    & (mod_counts_UMAP_df_focused['UMAP2']>= 3) & (mod_counts_UMAP_df_focused['UMAP2']<= 6)]

# Check
mod_counts_UMAP_df_focused_C8.head()

# Save those contig IDs
mod_counts_UMAP_df_focused_C8.index.to_list()

df_mod_counts_C8 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_focused_C8.index)]

In [ ]:
# Save contig IDs
mod_counts_C8_contigs = df_mod_counts_C8.index.to_list()

directory = 'UMAP_HAMBI_above100'
file_path = os.path.join(directory, 'mod_counts_C8_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C8_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 100 mod counts
#### C9

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_focused_C9 = mod_counts_UMAP_df_focused.loc[(mod_counts_UMAP_df_focused['UMAP1']>= 15) & (mod_counts_UMAP_df_focused['UMAP1']<= 16)
    & (mod_counts_UMAP_df_focused['UMAP2']>= 2) & (mod_counts_UMAP_df_focused['UMAP2']<= 4)]

# Check
mod_counts_UMAP_df_focused_C9.head()

# Save those contig IDs
mod_counts_UMAP_df_focused_C9.index.to_list()

df_mod_counts_C9 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_focused_C9.index)]

In [ ]:
# Save contig IDs
mod_counts_C9_contigs = df_mod_counts_C9.index.to_list()

directory = 'UMAP_HAMBI_above100'
file_path = os.path.join(directory, 'mod_counts_C9_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C9_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 100 mod counts
#### C10

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_focused_C10 = mod_counts_UMAP_df_focused.loc[(mod_counts_UMAP_df_focused['UMAP1']>= 16) & (mod_counts_UMAP_df_focused['UMAP1']<= 16.55)
    & (mod_counts_UMAP_df_focused['UMAP2']>= 2.8) & (mod_counts_UMAP_df_focused['UMAP2']<= 4)]

# Check
mod_counts_UMAP_df_focused_C10.head()

# Save those contig IDs
mod_counts_UMAP_df_focused_C10.index.to_list()

df_mod_counts_C10 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_focused_C10.index)]

In [ ]:
# Save contig IDs
mod_counts_C10_contigs = df_mod_counts_C10.index.to_list()

directory = 'UMAP_HAMBI_above100'
file_path = os.path.join(directory, 'mod_counts_C10_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C10_contigs:
        file.write(f"{item}\n")

## Extract clusters and redraw UMAP with subset of data
### > 100 mod counts
#### C11

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
mod_counts_UMAP_df_focused_C11 = mod_counts_UMAP_df_focused.loc[(mod_counts_UMAP_df_focused['UMAP1']>= 16.9) & (mod_counts_UMAP_df_focused['UMAP1']<= 17.35)
    & (mod_counts_UMAP_df_focused['UMAP2']>= 2.2) & (mod_counts_UMAP_df_focused['UMAP2']<= 3.2)]

# Check
mod_counts_UMAP_df_focused_C11.head()

# Save those contig IDs
mod_counts_UMAP_df_focused_C11.index.to_list()

df_mod_counts_C11 = df_mod_counts[df_mod_counts.index.isin(mod_counts_UMAP_df_focused_C11.index)]

In [ ]:
# Save contig IDs
mod_counts_C11_contigs = df_mod_counts_C11.index.to_list()

directory = 'UMAP_HAMBI_above100'
file_path = os.path.join(directory, 'mod_counts_C11_contigs.txt')

with open(file_path, 'w') as file:
    for item in mod_counts_C11_contigs:
        file.write(f"{item}\n")